In [3]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 12.6 MB/s 
     |████████████████████████████████| 453 kB 33.8 MB/s 


In [3]:
import random
import scipy as sp 
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Hannanum
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 소리는 무조건 '소리'

In [4]:
dic = {'tv': [62],   # 텔레비전, 티비
 '가방': [24, 26],
 '가위': [76],  # 문구
 '간식': [52, 53, 54, 55],
 '강아지': [16], # 견종, 멍멍이, 멍멍
 '개': [16],
 '거실': [57],
 '경마': [17],
 '고속열차': [6],
 '고양이': [15],    # 야옹이, 야옹
 '곰': [21],
 '곰돌이': [21],
 '곰인형': [77],
 '공': [32],
 '공항': [4],
 '과일': [46, 47, 49],
 '교통': [9],
 '그릇': [45],
 '글라스': [40],
 '기린': [23],
 '기차': [6],
 '까치': [14],
 '꽃': [58, 75],
 '꽃병': [75],
 '나이프': [43],
 '냉장고': [72],
 '넥타이': [27],
 '노트북': [63],
 '당근': [51],
 '대중교통': [5],
 '도넛': [54],
 '도로': [9, 81],
 '도서관': [73],
 '독서': [73],
 '동물': [15, 16, 17, 18, 19, 20, 21, 22, 23],
 '드라이기': [78],
 '디저트': [54, 55],
 '라켓': [38],
 '랩탑': [63],
 '리모컨': [65],
 '리모콘': [65],
 '마우스': [64],
 '말': [17],
 '목장': [19],
 '물병': [39],
 '물컵': [41],
 '미니백': [26],
 '바나나': [46],
 '밥그릇': [45],
 '방망이': [34],
 '배': [8],
 '배낭': [24],
 '배달': [3],
 '배트': [34],
 '백': [26],
 '백팩': [24],
 '버스': [5],
 '벤치': [13],
 '변기': [61],
 '변기통': [61],
 '병': [39],
 '병류': [39],
 '보드': [31, 36],
 '보트': [8],
 '볼': [32, 45],
 '부엌': [71, 72],
 '북극곰': [21],
 '브로콜리': [50],
 '비둘기': [14],
 '비행기': [4],
 '빵': [48, 54, 55],
 '사과': [47],
 '사람': [0],
 '사람들': [0],
 '새': [14],
 '샌드위치': [48],
 '생일': [55],
 '서프': [37],
 '서프보드': [37],
 '서핑': [37],
 '세면대': [61],
 '소': [19],
 '소떼': [19],
 '소파': [57],
 '소화기': [10],
 '소화전': [10],
 '숄더백': [26],
 '수저': [44],
 '숟가락': [44],
 '술병': [39],
 '술잔': [40],
 '스노우보드': [31],
 
 '스케이트': [36],
 '스케이트보드': [36],
 '스키': [30],
 '스키보드': [30],
 '스키장': [30, 31],
 '스포츠공': [32],
 '스푼': [44],
 '승마': [17],
 '승마장': [17],
 '승용차': [2],
 '시계': [74],
 '식기류': [42],
 '식사': [40, 41, 42, 43, 44, 45, 48, 52, 53, 60],
 '식탁': [60],
 '신호등': [9],
 '싱크': [71],
 '싱크대': [71],
 '안방': [59],
 '야구': [34, 34, 35],
 '야구방망이': [34],
 '야구장갑': [35],
 '야채': [50, 51],
 '양': [18],
 '양떼': [18],
 '양치': [79],
 '얼룩말': [22],
 '여행': [28],
 '여행가방': [28],
 '연': [33],
 '연날리기': [33],
 '열차': [6],
 '오렌지': [49],
 '오븐': [69],
 '오토바이': [3],
 '와인잔': [40],
 '요리': [50, 51, 68, 69],
 '요트': [8],
 '우산': [25],
 '운동': [32, 34, 35, 36, 37],
 '운전': [2],
 '원반': [29],
 '원반던지기': [29],
 '유리컵': [41],
 '음메': [19],
 '음식': [45, 48, 52, 53, 54, 55],
 '의자': [13, 56],
 '인형': [77],
 '일': [63],
 '자동차': [2],
 '자전거': [1],
 '작업': [63],
 '잠': [59],
 '장갑': [35],
 '전동차': [3],
 '전자레인지': [68],
 '전화' : [67],
 '접시': [45],
 '정지': [11],
 '정지 신호': [11],
 '조류': [14],
 '주스병': [39],
 '주차': [12],
 '주차 미터기': [12],
 '주차장': [12],
 '차': [2],
 '차량': [2],
 '참새': [14],
 '채소': [50, 51],
 '책': [73],
 '책방': [73],
 '책장': [73],
 '출국': [28],
 '치약': [79],
 '침대': [59],
 '침실': [59],
 '칫솔': [79],
 '칼': [43],
 '캐리어': [28],
 '컴퓨터': [63],
 '컵': [41],
 '케이크': [55],
 '코끼리': [20],
 '키보드': [66],
 '타이': [27],
 '타자': [34],
 '테니스': [38],
 '테니스 라켓': [38],
 '테디베어': [77],
 '토스터': [70],
 '토스트': [48, 70],
 '토스트기': [70],
 '투수': [35],
 '트럭': [7],
 '포수': [35],
 '포크': [42],
 '핸드폰': [67], #폰, #휴대폰
 '스마트폰': [67],

 '프리스비': [29],
 '프리즈비': [29],
 '피자': [53],
 '핫도그': [52],
 '항공': [4],
 '해양스포츠': [37],
 '핸드백': [26],
 
 '화병': [75],
 '화분': [58],
 '화장실': [61, 79],
 '화초': [58],
 


 

 '벨소리' : [80],
 '알람소리' : [80],
 '경적소리' : [81],
 '박수소리' : [82],
 '아기울음소리' : [83],
 '드럼소리' : [84],
 '기타소리' : [85],
 '피아노소리' : [86],
 '악기소리' : [84, 85, 86],
 '사자소리' : [87],
 '총소리' : [88],
 '빗소리' : [89],
 '비명소리' : [90],
 '사이렌소리' : [91],
 '코고는소리' : [92],
 '웃음소리' : [93],
 '천둥소리' : [94],
 '청소기소리' : [95],


 '비행기소리' : [96],
 '새소리' : [97],
 '강아지소리' : [98],
 '고양이소리' : [99],
 
 


#  'others' : [1000],
 
 
 }

# [1] 데이터 읽어오기

In [6]:
od_raw = pd.read_csv('./flanders_i.csv', header=None, index_col=False, names=['drop', 'start', 'end', 'object_nums', 4, 5, 6, 7])
# od_raw = pd.read_csv('/content/drive/MyDrive/DRIVE_ds_study/Vmark/flanders_i.csv', header=None, index_col=False, names=['drop', 'start', 'end', 'object_nums', 4, 5, 6, 7])
od_raw.drop(['drop'], axis=1, inplace=True)
od_raw

# n = 자르는 초단위
n = 2
detector = []
for time in tqdm(range(0, int(od_raw.iloc[-1,1] + n), n)):
# for time in tqdm(range(0, 100, n)):
    od_tmp = []
    for i in range(len(od_raw)) :
        if (time < od_raw.iloc[i,0] <= time+n) or (time < od_raw.iloc[i,1] <= time+n):
            dect_num = int(od_raw.iloc[i,2].split('_')[0])
            for key, val in dic.items():
                # print(key, val, dect_num)
                if val.count(dect_num) != 0:
                    # print(f'{time} ~ {time+3} : {key}')
                    od_tmp.append(key)
    if od_tmp == []:
        continue
    else:
        detector.append({
        "start" : time,
        "end" : time+n,
        "detect" : ' '.join(list(set(od_tmp)))
        })



df_detector = pd.DataFrame(detector)
df_detector

C:\Users\user\AppData\Local\Temp/ipykernel_30944/1191260167.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  od_raw = pd.read_csv('./flanders_i.csv', header=None, index_col=False, names=['drop', 'start', 'end', 'object_nums', 4, 5, 6, 7])
100%|██████████| 271/271 [00:04<00:00, 61.21it/s]


,start,end,detect
0,70,72,사람 사람들
1,74,76,핸드폰 스마트폰 전화
2,80,82,핸드폰 스마트폰 전화
3,84,86,핸드폰 스마트폰 전화
4,86,88,핸드폰 스마트폰 전화
...,...,...,...
176,530,532,식사 컵 유리컵 치약 물컵 개 강아지 칫솔 화장실 양치 동물
177,534,536,강아지 개 테디베어 인형 곰인형 동물
178,536,538,도서관 책장 독서 사람 책방 책 사람들
179,538,540,사람 사람들


In [ ]:
sd_raw = pd.read_csv('./movie2_df.csv', index_col=0)

n = 2
# detector = []
for time in tqdm(range(0, int(sd_raw.iloc[-1,1] + n), n)):
# for time in tqdm(range(0, 100, 3)):
    sd_tmp = []
    for i in range(len(sd_raw)) :
        if (time < sd_raw.iloc[i,0] <= time+n) or (time < sd_raw.iloc[i,1] <= time+n):
            dect_num = sd_raw.iloc[i,2]
            for key, val in dic.items():
                # print(key, val, dect_num)
                if val.count(dect_num) != 0:
                    # print(f'{time} ~ {time+3} : {key}')
                    sd_tmp.append(key)
    if sd_tmp == []:
        continue
    else:
        sd_tmp = ' '.join(list(set(sd_tmp)))
        df_detector.iloc[int(((time+n)/2)-1), 2] += ' ' + sd_tmp


df_detector


 12%|█▏        | 49/418 [00:02<00:16, 22.81it/s]


IndexError: single positional indexer is out-of-bounds

In [7]:
contents_for_vectorize = []
for idx, row in df_detector.iterrows():
    contents_for_vectorize.append(row[2])

stop_words = ['소리', '장면']
vectorizer = TfidfVectorizer(min_df=1, decode_error='ignore', stop_words=stop_words)
X = vectorizer.fit_transform(contents_for_vectorize)

In [8]:
url_dict = {"flanders_original": "https://www.youtube.com/watch?v=CTbvsB0K0L4&t=",
            "flanders_yolo" :  "https://www.youtube.com/watch?v=SPo5CemLLZY&t=",
            "sesang_original": "https://www.youtube.com/watch?v=try6rRGsdrw&t=",
            "sesang_yolo" :  "https://www.youtube.com/watch?v=VskOZglJiio&t=",}

---

### 1안 (기존) : 그냥 토큰화

In [9]:
input = '비행기가 날아가는 장면'

In [ ]:
new_post_tokens = [noun for noun in hannanum.nouns(input) if noun != '장면' and '소리']
new_post_tokens

['비행기']

In [10]:
# 1안 (기존) : 그냥 토큰화

hannanum = Hannanum()
# input = '사람과 가방이 나오는 장면'
new_post_tokens = [noun for noun in hannanum.nouns(input) if noun != '장면' and '소리']
new_post_for_vectorize = [' '.join(new_post_tokens)]
print(f'input query의 토큰화 ==> {new_post_for_vectorize}')
new_post_vec = vectorizer.transform(new_post_for_vectorize)

def norm_cos_sim(v1, v2):
    v1_normalized = v1 / sp.linalg.norm(v1.toarray())
    v2_normalized = v2 / sp.linalg.norm(v2.toarray())
    delta = cosine_similarity(v1_normalized, v2_normalized)
    return sp.linalg.norm(delta) 
    # return sp.linalg.norm(delta.toarray())

dist = [norm_cos_sim(each, new_post_vec) for each in X]


print('-'*50)
print(f'Best post is {dist.index(max(dist))}, dist = {max(dist)}')
print(f'input query ==> "{input}"')
print(f'Test post is ==> {new_post_tokens}')
print()
print(f"""Best dist post is ==> {df_detector['start'][dist.index(max(dist))]}초 ~ {df_detector['end'][dist.index(max(dist))]}초
                                : {contents_for_vectorize[dist.index(max(dist))]}""");

input query의 토큰화 ==> ['비행기']
--------------------------------------------------
Best post is 107, dist = 0.4158558791882202
input query ==> "비행기가 날아가는 장면"
Test post is ==> ['비행기']

Best dist post is ==> 368초 ~ 370초
                                : 비둘기 공항 까치 새 조류 항공 비행기 참새


### 2안 : dic에서 가져오기

In [ ]:
# 2안 : dic에서 가져오기

hannanum = Hannanum()
# input = '식탁에서 마이크로 노래부르는 장면'
tokens = [token for token in hannanum.nouns(input) if token != '장면' and '소리']
new_post_tokens = []

for token in tokens:
    tmp = []
    for key, val in dic.items():
        try:
            if token == key:
                dect_num = val

                for n in dect_num :
                    for key2, val2 in dic.items():
                        if val2.count(n) != 0:
                            tmp.append(key2)
        except:
            pass

    if tmp == []:
        continue
    else:
        for i in range(len(tmp)):
            new_post_tokens.append(tmp[i])


def norm_cos_sim(v1, v2):
    v1_normalized = v1 / sp.linalg.norm(v1.toarray())
    v2_normalized = v2 / sp.linalg.norm(v2.toarray())
    delta = cosine_similarity(v1_normalized, v2_normalized)
    return sp.linalg.norm(delta) 
    # return sp.linalg.norm(delta.toarray())

dist = [norm_cos_sim(each, new_post_vec) for each in X]


print('-'*50)
print(f'Best post is {dist.index(max(dist))}, dist = {max(dist)}')
print(f'input query ==> "{input}"')
print(f'Test post is ==> {new_post_tokens}')
print()
print(f"""Best dist post is ==> {df_detector['start'][dist.index(max(dist))]}초 ~ {df_detector['end'][dist.index(max(dist))]}초
                                : {contents_for_vectorize[dist.index(max(dist))]}""");

--------------------------------------------------
Best post is 107, dist = 0.4158558791882202
input query ==> "비행기가 날아가는 장면"
Test post is ==> ['공항', '비행기', '항공']

Best dist post is ==> 368초 ~ 370초
                                : 비둘기 항공 까치 새 비행기 참새 조류 공항


### 3안 : dic에서 단일 key 값은 그것만 가져오기

In [11]:
# 3안 : dic에서 단일 key 값은 그것만 가져오기

hannanum = Hannanum()
# input = '식탁에서 마이크로 노래부르는 장면'
tokens = [token for token in hannanum.nouns(input) if token != '장면' and '소리']

for token in tokens:
    tmp = []
    for key, val in dic.items():
        try:
            if token == key and len(val) == 1:
                tmp.append(key)

            elif token == key and len(val) != 1:
                dect_num = val
                for n in dect_num :
                    for key2, val2 in dic.items():
                        if val2.count(n) != 0:
                            tmp.append(key2)
        except:
            pass

    if tmp == []:
        continue
    else:
        for i in range(len(tmp)):
            new_post_tokens.append(tmp[i])
new_post_tokens

new_post_tokens = list(set(new_post_tokens))
new_post_for_vectorize = [' '.join(new_post_tokens)]
print(f'input query의 토큰화 ==> {new_post_for_vectorize}')
new_post_vec = vectorizer.transform(new_post_for_vectorize)


def norm_cos_sim(v1, v2):
    v1_normalized = v1 / sp.linalg.norm(v1.toarray())
    v2_normalized = v2 / sp.linalg.norm(v2.toarray())
    delta = cosine_similarity(v1_normalized, v2_normalized)
    return sp.linalg.norm(delta) 
    # return sp.linalg.norm(delta.toarray())

dist = [norm_cos_sim(each, new_post_vec) for each in X]


print('-'*50)
print(f'Best post is {dist.index(max(dist))}, dist = {max(dist)}')
print(f'input query ==> "{input}"')
print(f'Test post is ==> {new_post_tokens}')
print()
df_detector_rank = df_detector.copy
print(f"""[TOP 5] dist post First is ==> {df_detector['start'][dist.index(max(dist))]}초 ~ {df_detector['end'][dist.index(max(dist))]}초 || {contents_for_vectorize[dist.index(max(dist))]}""");
print(f"""[TOP 5] dist post Second is ==> {df_detector['start'][dist.index(max(dist))]}초 ~ {df_detector['end'][dist.index(max(dist))]}초 || {contents_for_vectorize[dist.index(max(dist))]}""");


input query의 토큰화 ==> ['비행기']
--------------------------------------------------
Best post is 107, dist = 0.4158558791882202
input query ==> "비행기가 날아가는 장면"
Test post is ==> ['비행기']

[TOP 5] dist post First is ==> 368초 ~ 370초 || 비둘기 공항 까치 새 조류 항공 비행기 참새
[TOP 5] dist post Second is ==> 368초 ~ 370초 || 비둘기 공항 까치 새 조류 항공 비행기 참새


In [12]:
# 3안 : dic에서 단일 key 값은 그것만 가져오기

hannanum = Hannanum()
# input = '식탁에서 마이크로 노래부르는 장면'
tokens = [token for token in hannanum.nouns(input) if token != '장면' and '소리']

for token in tokens:
    tmp = []
    for key, val in dic.items():
        try:
            if token == key and len(val) == 1:
                tmp.append(key)

            elif token == key and len(val) != 1:
                dect_num = val
                for n in dect_num :
                    for key2, val2 in dic.items():
                        if val2.count(n) != 0:
                            tmp.append(key2)
        except:
            pass

    if tmp == []:
        continue
    else:
        for i in range(len(tmp)):
            new_post_tokens.append(tmp[i])
new_post_tokens

new_post_tokens = list(set(new_post_tokens))
new_post_for_vectorize = [' '.join(new_post_tokens)]
print(f'input query의 토큰화 ==> {new_post_for_vectorize}')
new_post_vec = vectorizer.transform(new_post_for_vectorize)


def norm_cos_sim(v1, v2):
    v1_normalized = v1 / sp.linalg.norm(v1.toarray())
    v2_normalized = v2 / sp.linalg.norm(v2.toarray())
    delta = cosine_similarity(v1_normalized, v2_normalized)
    return sp.linalg.norm(delta) 
    # return sp.linalg.norm(delta.toarray())

dist = [norm_cos_sim(each, new_post_vec) for each in X]

dist_rank = [(val, idx) for idx, val in enumerate(dist)]
dist_rank.sort(reverse=True)

print('-'*50)
print(f'Best post is {dist.index(max(dist))}, dist = {max(dist)}')
print(f'input query ==> "{input}"')
print(f'Test post is ==> {new_post_tokens}')
print()
# df_detector_rank = df_detector.copy
print(f"""[TOP 5] First is  ==> {df_detector['start'][dist_rank[0][1]]}초 ~ {df_detector['end'][dist_rank[0][1]]}초 || {contents_for_vectorize[dist_rank[0][1]]}
        origin  : {url_dict['flanders_original']}{int(df_detector['start'][dist_rank[0][1]])}s
        yolo    : {url_dict['flanders_yolo']}{int(df_detector['start'][dist_rank[0][1]])}s""");
print(f"""[TOP 5] Second is ==> {df_detector['start'][dist_rank[1][1]]}초 ~ {df_detector['end'][dist_rank[1][1]]}초 || {contents_for_vectorize[dist_rank[1][1]]}
        origin  : {url_dict['flanders_original']}{int(df_detector['start'][dist_rank[1][1]])}s
        yolo    : {url_dict['flanders_yolo']}{int(df_detector['start'][dist_rank[1][1]])}s""");
print(f"""[TOP 5] Third is  ==> {df_detector['start'][dist_rank[2][1]]}초 ~ {df_detector['end'][dist_rank[2][1]]}초 || {contents_for_vectorize[dist_rank[2][1]]}
        origin  : {url_dict['flanders_original']}{int(df_detector['start'][dist_rank[2][1]])}s
        yolo    : {url_dict['flanders_yolo']}{int(df_detector['start'][dist_rank[2][1]])}s""");
print(f"""[TOP 5] Fourth is ==> {df_detector['start'][dist_rank[3][1]]}초 ~ {df_detector['end'][dist_rank[3][1]]}초 || {contents_for_vectorize[dist_rank[3][1]]}
        origin  : {url_dict['flanders_original']}{int(df_detector['start'][dist_rank[3][1]])}s
        yolo    : {url_dict['flanders_yolo']}{int(df_detector['start'][dist_rank[3][1]])}s""");
print(f"""[TOP 5] Fifth is  ==> {df_detector['start'][dist_rank[4][1]]}초 ~ {df_detector['end'][dist_rank[4][1]]}초 || {contents_for_vectorize[dist_rank[4][1]]}
        origin  : {url_dict['flanders_original']}{int(df_detector['start'][dist_rank[4][1]])}s
        yolo    : {url_dict['flanders_yolo']}{int(df_detector['start'][dist_rank[4][1]])}s""");


input query의 토큰화 ==> ['비행기']
--------------------------------------------------
Best post is 107, dist = 0.4158558791882202
input query ==> "비행기가 날아가는 장면"
Test post is ==> ['비행기']

[TOP 5] First is  ==> 368초 ~ 370초 || 비둘기 공항 까치 새 조류 항공 비행기 참새
        origin  : https://www.youtube.com/watch?v=CTbvsB0K0L4&t=368s
        yolo    : https://www.youtube.com/watch?v=SPo5CemLLZY&t=368s
[TOP 5] Second is ==> 218초 ~ 220초 || 공항 기차 열차 고속열차 소화기 사람 항공 소화전 비행기 사람들
        origin  : https://www.youtube.com/watch?v=CTbvsB0K0L4&t=218s
        yolo    : https://www.youtube.com/watch?v=SPo5CemLLZY&t=218s
[TOP 5] Third is  ==> 540초 ~ 542초 || 사람 사람들
        origin  : https://www.youtube.com/watch?v=CTbvsB0K0L4&t=540s
        yolo    : https://www.youtube.com/watch?v=SPo5CemLLZY&t=540s
[TOP 5] Fourth is ==> 538초 ~ 540초 || 사람 사람들
        origin  : https://www.youtube.com/watch?v=CTbvsB0K0L4&t=538s
        yolo    : https://www.youtube.com/watch?v=SPo5CemLLZY&t=538s
[TOP 5] Fifth is  ==> 536초 ~ 538초 || 도서관 책장 

In [ ]:
dist_rank = [(val, idx) for idx, val in enumerate(dist)]

In [ ]:
dist_rank.sort(reverse=True)
dist_rank
# dist_rank[0][1]

[(0.7202835113802224, 107),
 (0.6482527695809011, 47),
 (0.0, 180),
 (0.0, 179),
 (0.0, 178),
 (0.0, 177),
 (0.0, 176),
 (0.0, 175),
 (0.0, 174),
 (0.0, 173),
 (0.0, 172),
 (0.0, 171),
 (0.0, 170),
 (0.0, 169),
 (0.0, 168),
 (0.0, 167),
 (0.0, 166),
 (0.0, 165),
 (0.0, 164),
 (0.0, 163),
 (0.0, 162),
 (0.0, 161),
 (0.0, 160),
 (0.0, 159),
 (0.0, 158),
 (0.0, 157),
 (0.0, 156),
 (0.0, 155),
 (0.0, 154),
 (0.0, 153),
 (0.0, 152),
 (0.0, 151),
 (0.0, 150),
 (0.0, 149),
 (0.0, 148),
 (0.0, 147),
 (0.0, 146),
 (0.0, 145),
 (0.0, 144),
 (0.0, 143),
 (0.0, 142),
 (0.0, 141),
 (0.0, 140),
 (0.0, 139),
 (0.0, 138),
 (0.0, 137),
 (0.0, 136),
 (0.0, 135),
 (0.0, 134),
 (0.0, 133),
 (0.0, 132),
 (0.0, 131),
 (0.0, 130),
 (0.0, 129),
 (0.0, 128),
 (0.0, 127),
 (0.0, 126),
 (0.0, 125),
 (0.0, 124),
 (0.0, 123),
 (0.0, 122),
 (0.0, 121),
 (0.0, 120),
 (0.0, 119),
 (0.0, 118),
 (0.0, 117),
 (0.0, 116),
 (0.0, 115),
 (0.0, 114),
 (0.0, 113),
 (0.0, 112),
 (0.0, 111),
 (0.0, 110),
 (0.0, 109),
 (0.0, 10

In [ ]:
url_dict['flanders_original']

'https://www.youtube.com/watch?v=CTbvsB0K0L4='